# Distributed modeling with static schedule

In [ ]:
using Distributed, PyPlot

In [ ]:
addprocs(2)

In [ ]:
@everywhere using DistributedArrays, DistributedJets, Jets, JetPackWave, Random, Wave

In [ ]:
v = read!("../02_marmousi_model_setup/marmousi_vp_20m_176x851.bin", Array{Float32}(undef, 176,851));
dz,dx = 20.0,20.0
nz,nx = size(v)
@show dz,dx
@show nz,nx;

In [ ]:
sx = range(0,length=4,stop=(851-1)*20)
nshots = length(sx)
@show nshots;

In [ ]:
@everywhere scratchdir = "/scratch"

In [ ]:
@everywhere function makeF(i,sx)
    nz,nx,dz,dx = 176,851,20.0,20.0    
    JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        ntrec = 1101,
        dtrec = 0.002,
        dtmod = 0.001,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=5.0),
        sx = sx[i],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        srcfieldfile = "$scratchdir/field-$i.$(randstring()).bin",
        reportinterval=1000)
end

In [ ]:
F = @blockop DArray(I->[makeF(i,sx) for i in I[1], j in I[2]], (nshots,1))

In [ ]:
d = F*v

## Look at shot 4

In [ ]:
ishot = 2
trcs = getblock(d,ishot)
imshow(trcs,aspect="auto",cmap="gray");clim(-1,1);colorbar();

## Remove source field files on local disk

In [ ]:
close(F)

## Remove workers

In [ ]:
rmprocs(workers())